<a href="https://colab.research.google.com/github/caseynjustus/document-ai-samples/blob/main/colabs/CM360/%5Bcolab_2%5D_Vision_API_for_Creative_Parsing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The Project focuses on analysing Ad Creatives and its related performance metrics and tries to find correlations between the elements present in the past creatives to suggest what can be good add-ons and what can help to improve performance of future creatives.

Install Python Dependencies

In [ ]:
!pip install ndjson google-cloud-vision==2.0.0 selenium google-cloud-videointelligence==2.0.0  protobuf==3.14.0

     |████████████████████████████████| 460kB 24.6MB/s 
     |████████████████████████████████| 911kB 38.5MB/s 
     |████████████████████████████████| 184kB 40.1MB/s 
     |████████████████████████████████| 1.0MB 43.1MB/s 
     |████████████████████████████████| 51kB 5.3MB/s 
     |████████████████████████████████| 522kB 42.5MB/s 
     |████████████████████████████████| 645kB 32.7MB/s 
  Found existing installation: protobuf 3.12.4
    Uninstalling protobuf-3.12.4:
      Successfully uninstalled protobuf-3.12.4
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Install Selenium Driver

In [ ]:
!apt-get update && apt install chromium-chromedriver

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

### Input1
Paste the json key downloaded from GCP in cell below

In [ ]:
%%writefile credentials.json
{
  "type": "service_account",
  "project_id": "[project_id]",
  "private_key_id": "[private_key_id]",
  "private_key": "[private_key]",
  "client_email": "[client_email]",
  "client_id": "[client_id]",
  "auth_uri": "https://accounts.google.com/o/oauth2/auth",
  "token_uri": "https://oauth2.googleapis.com/token",
  "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
  "client_x509_cert_url": "[client_x509_cert_url]"
  }

### Input2

Update Values in params below to specify the input dataset for urls, output dataset and other key params

In [ ]:
%%writefile utils.py

from google.oauth2 import service_account
from google.cloud import storage
from google.cloud import bigquery
from google.cloud import vision
from selenium import webdriver
from google.cloud import videointelligence

job_type = 'image' #@param [ "image", "video"]
gcp_project_id =  "oculi-v2-dev" #@param {type:'string'}
bq = "bq" #@param [ "bq", "gcs"]
input_dataset_name="demoverse" #@param {type:'string'}
input_table_name="creative_urls" #@param {type:'string'}
gcs_bucket_name="demoverse" #@param {type:'string'}
output_dataset_name="demoverse" #@param {type:'string'}
# limit= '' #@param {type:'string'}

#bq_input_schema: row_num, creative_id, file_name (should include extension), full_url, creative_pixel_size (recommended) 
def get_params():
  params = {'job_type': job_type, #image, video
            'gcp_project': gcp_project_id, 
            'asset_source': bq , #bq, gcs
            'bq_input_dataset': input_dataset_name, 
            'bq_input_tablename': input_table_name, #static_urls, html_urls, animated_gif_urls, video_urls
            'gcs_bucket_name': gcs_bucket_name, #gcs-start-images-oculi-final, bq-start-oculi-final
            'bq_output_dataset': output_dataset_name,
            'image_config': {
              'image_size_included': True, # static_image_urls_superset, video_urls2 - False | else True 
              'html5': {
                  'every_n_seconds': 1,
                  'max_animation_time': 7, #in seconds
                  'chromedriver_path': 'chromedriver'
              },
              'gif':{
                  'every_n_frames': 'all' # / 'all' / 'default' -f & l / 5 (any number)
              }
             }
            }
  return params

def validate_params():
  global params
  project_id = params['gcp_project']
  input_dataset = params['bq_input_dataset']
  input_table = params['bq_input_tablename'] 
  gcs_bucket_name = params['gcs_bucket_name']
  output_dataset = params['bq_output_dataset']

  def exists_bq_dataset(project_id, dataset_name):
    pass
  
  def exists_bq_table(project_id, dataset_name, table_name):
    pass

  def exists_gcs_bucket(project_id, bucket_name):
    pass

  def create_bq_dataset(project_id, dataset_name):
    pass

  def create_gcs_bucket(project_id, bucket_name):
    pass

  if params['asset_source'] == "bq":
    if not (exists_bq_dataset() and exists_bq_table()):
      return "Source BQ dataset and/or table not found."
    if not exists_gcs_bucket():
      pass
      


def get_credentials():
  credentials = service_account.Credentials.from_service_account_file(
    'credentials.json', scopes=["https://www.googleapis.com/auth/cloud-platform"])
  return credentials

def init():
  params = get_params()
  credentials = get_credentials()
  bq_client = bigquery.Client(credentials=credentials, project=credentials.project_id)
  storage_client = storage.Client(credentials=credentials,  project=credentials.project_id)
  gcs_bucket = storage_client.get_bucket(params['gcs_bucket_name'])
  return params, credentials, bq_client, storage_client, gcs_bucket

def init_webdriver():
  global params
  chromedriver_path = params['image_config']['html5']['chromedriver_path']
  if chromedriver_path:
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(chromedriver_path, options=chrome_options)
    return driver

def get_image_client():
  global credentials
  vision_client = vision.ImageAnnotatorClient(credentials=credentials)
  return vision_client

def get_video_api_params():
  global credentials
  vid_client = videointelligence.VideoIntelligenceServiceClient(
        credentials=credentials)
  vid_config = videointelligence.SpeechTranscriptionConfig(
        language_code="en-US", enable_automatic_punctuation=True)
  video_context = videointelligence.VideoContext(
      speech_transcription_config=vid_config)
  vid_features = ['LABEL_DETECTION', 'TEXT_DETECTION', 'SPEECH_TRANSCRIPTION', 
              'EXPLICIT_CONTENT_DETECTION', 'SHOT_CHANGE_DETECTION',
              'OBJECT_TRACKING']
  return vid_client, vid_config, video_context, vid_features

params, credentials, bq_client, storage_client, gcs_bucket = init()
vision_client = get_image_client()
driver = init_webdriver()
vid_client, vid_config, video_context, vid_features = get_video_api_params()

Overwriting utils.py


In [ ]:
%%writefile bq_helper.py

from utils import params, bq_client
from google.cloud import bigquery
import ndjson

#bq_input_schema: row_num, creative_id, file_name (should include extension), full_url, creative_pixel_size (recommended) 
def read_from_bq(sql):
  global bq_client
  global params
  query_job = bq_client.query(sql)
  results = query_job.result()
  input_rows = []
  job_type = params['job_type']
  image_size_included = params['image_config']['image_size_included']
  for r in results:
    row = {'row_num': r.row_num,
           'creative_id': r.creative_id,
           'file_name': r.file_name, 
           'full_url': r.full_url}
    if job_type == "image" and image_size_included:
      row['creative_pixel_size'] = r.creative_pixel_size
    input_rows = input_rows + [row]
  return input_rows


def write_to_bq(dataset_name, table_name, table_data):
  try:
    global params, bq_client
    table_file_name = table_name +'.json'
    with open(table_file_name, 'w') as f:
      ndjson.dump(table_data, f)
    dataset_ref = bq_client.dataset(dataset_name)
    table_ref = dataset_ref.table(table_name)
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = 'WRITE_TRUNCATE'
    job_config.source_format = bigquery.SourceFormat.NEWLINE_DELIMITED_JSON
    job_config.autodetect = True
    with open(table_file_name, "rb") as source_file:
      job = bq_client.load_table_from_file(source_file, table_ref,
                                           job_config=job_config)
      job.result()
    print(f"Loaded {table_name} table")
  except Exception as e:
    print(f"ERROR - {table_name}: {e}")


def filter_output(metadata, endpoint):
  global params
  job_type = params["job_type"]
  if job_type == "image":
    frames = metadata['frames']
    process_frame = lambda f: {'frame_id': f['frame_id'],
                              'frame_url': f['frame_url'],
                                endpoint: f[endpoint]}
    frames = [process_frame(f) for f in frames]
    filtered_output = {
        "row_num": metadata['row_num'],
        "creative_id": metadata["creative_id"],
        "gcs_url": metadata["gcs_url"],
        "frames": frames}
  else:
    filtered_output = {
        "creative_id": metadata["creative_id"],
        "creative_url": metadata["creative_url"],
         endpoint: metadata[endpoint]}
  return filtered_output


def get_endpoints(job_type):
  image_endpoints = ['image_properties_annotation', 'text_annotations',
                     'safe_search_annotation', 'label_annotations',
                     'logo_annotations', 'face_annotations',
                     'localized_object_annotations']
  video_endpoints = ['text_annotations', 'segment_label_annotations',
                     'shot_label_annotations', 'speech_transcriptions',
                     'explicit_annotation', 'object_annotations',
                     'shot_annotations']
  if job_type == "image":
    return image_endpoints
  else:
    return video_endpoints

Overwriting bq_helper.py


In [ ]:
%%writefile gcs_helper.py

from utils import params, storage_client, gcs_bucket, driver
import requests
import re
from PIL import Image
from io import BytesIO
import time
import base64
import math
import datetime
from selenium import webdriver


def upload_content(content, job_type, file_name, frame_name=None):
  global params
  global gcs_bucket
  """
  with open(frame_name, "wb") as f:
      f.write(content)
  """
  frame_path = ""
  if frame_name:
    frame_path = f"/frames/{frame_name}"
    file_name = file_name.replace(".","_")
  gcs_file_name = f"{job_type}/{file_name}{frame_path}"
  blob = gcs_bucket.blob(gcs_file_name)
  content_type = "image/jpg" if job_type == "image" else "video/mp4"
  if not blob.exists():
    blob.upload_from_string(data=content, content_type=content_type)
  gcs_url = f"gs://{params['gcs_bucket_name']}/{gcs_file_name}"
  return gcs_url


def get_html_frames(gcs_obj, base_file_name, url, image_size=''):
  global params
  #global driver
  row_num = gcs_obj['row_num']
  frames = []
  sleep_time = params['image_config']['html5']['every_n_seconds']
  animation_time = params['image_config']['html5']['max_animation_time']
  num_loops = math.ceil(animation_time/sleep_time)
  # -------- 
  chromedriver_path = params['image_config']['html5']['chromedriver_path']
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome(chromedriver_path, options=chrome_options)
  # --- 
  if image_size:
    width, height = image_size.split('x')
    driver.set_window_size(width, height)
  time.sleep(1)
  driver.get(url)
  time.sleep(1)
  for i in range(num_loops):
    raw_content = driver.get_screenshot_as_base64()
    
    byte_str = base64.b64decode(raw_content)
    frame_id = f"{row_num}_f{i}"
    frame_fname = f"{frame_id}.png"
    frame_url = upload_content(byte_str, "image", base_file_name,
                               frame_fname)
    frame = {'frame_id':frame_id, 'frame_url': frame_url}
    frames = frames + [frame]
    screenshot_name = f"{frame_id}_screenshot.png"
    #driver.save_screenshot(screenshot_name)
    time.sleep(sleep_time)
  return frames


def get_gif_frames(gcs_obj, content, base_file_name):
  global params
  row_num, gcs_url = gcs_obj['row_num'], gcs_obj['gcs_url']
  frames = []
  image = Image.open(content)
  step = params['image_config']['gif']['every_n_frames']
  if step == "all":
    step = 1
  elif step == "default":
    step = image.n_frames-1 if image.n_frames > 1 else 1
  for i in range(0, image.n_frames, step):
    image.seek(i)
    imgByteArr = BytesIO()
    image.save(imgByteArr,'GIF')
    byte_str = imgByteArr.getvalue()
    frame_id = f"{row_num}_f{i}"
    frame_fname = f"{frame_id}.gif"
    frame_url = upload_content(byte_str, "image", base_file_name,
                             frame_fname)
    frame = {'frame_id':frame_id, 'frame_url': frame_url}
    frames = frames + [frame]
  return frames
    
#bq_input_schema: row_num, creative_id, file_name (should include extension), full_url
def read_from_gcs():
  global params
  global storage_client
  global gcs_bucket
  blobs = storage_client.list_blobs(gcs_bucket)
  metadata_rows = []
  for i, blob in enumerate(blobs):
    row_num, creative_id = i+1, i+1
    file_name = blob.name
    full_url = f"https://storage.cloud.google.com/{gcs_bucket.name}/{file_name}"
    metadata_row = {'row_num': row_num, 
                    'creative_id': creative_id, 
                    'file_name': file_name, 
                    'full_url': full_url}
    metadata_rows = metadata_rows + [metadata_row]
  return metadata_rows
  
def generate_signed_url(file_name):
  global gcs_bucket
  blob = gcs_bucket.blob(file_name)
  current_time = datetime.datetime.now()
  in_2_mins = current_time + datetime.timedelta(seconds=120)
  url = blob.generate_signed_url(in_2_mins)
  return url

def get_content(full_url):
  response = requests.get(full_url, verify=False)
  content = bytes(response.content)
  return content

def write_to_gcs(asset):
  try:
    creative_id, row_num = asset['creative_id'], asset['row_num']
    file_name, full_url, image_size = asset['file_name'], asset['full_url'], ''
    global params
    global storage_client
    global gcs_bucket
    job_type, gcs_bucket_name = params['job_type'], params['gcs_bucket_name']
    gcp_project = params['gcp_project']
    if job_type == "image" and params['image_config']['image_size_included']:
      image_size = asset['creative_pixel_size']
    is_html5 = lambda url: True if url[-4:] == "html" else False
    is_gif = lambda url: True if url[-3:] == "gif" else False
    is_static = lambda url: (job_type == "image" and not (is_html5(url) or is_gif(url)))
    gcs_obj = {'row_num': row_num,
               'creative_id': creative_id}
    file_url = full_url
    generated_gcs_url = False
    #----
    if params["asset_source"] == "gcs":
      gcs_url = f'gs://{full_url[33:]}'
      generated_gcs_url = True
      if job_type == "video":
        gcs_obj['gcs_url'] = gcs_url
        return gcs_obj
      if is_static(full_url):
        gcs_obj['gcs_url'] = full_url
        gcs_obj['frames'] = [{'frame_id': f"{row_num}_f0", 'frame_url': gcs_url}]
        return gcs_obj
      full_url = generate_signed_url(file_name)
    #-----
    if is_html5(file_url):
      gcs_prefix = f"https://console.cloud.google.com/storage/browser"
      base_gcs_url = f"{gcs_bucket_name}/{file_name}/frames".replace(".","_")
      gcs_url = f"{gcs_prefix}/{base_gcs_url}"
      gcs_obj['gcs_url'] = gcs_url
      frames = get_html_frames(gcs_obj, file_name, full_url, image_size)
      gcs_obj['frames'] = frames
      return gcs_obj

    if is_gif(file_url):
      content = get_content(full_url)
      if not generated_gcs_url:
        gcs_url = upload_content(content, job_type, file_name)
      gcs_obj['gcs_url'] = f"https://storage.cloud.google.com/{gcs_url[4:]}"
      bytesio_content = BytesIO(content)
      gcs_obj['frames'] = get_gif_frames(gcs_obj, bytesio_content, file_name)
      return gcs_obj
  
    if is_static(file_url):
      content = get_content(full_url)
      if not generated_gcs_url:
        gcs_url = upload_content(content, job_type, file_name)
      gcs_obj['gcs_url'] = f"https://storage.cloud.google.com/{gcs_url[4:]}"
      gcs_obj['frames'] = [{'frame_id': f"{row_num}_f0",
                            'frame_url': gcs_url}]
      return gcs_obj
    
    content = get_content(full_url)
    gcs_obj['gcs_url'] = upload_content(content, job_type, file_name)
    print(f"GCS Uploader: {row_num}")
    return gcs_obj
  except Exception as e:
    creative_id, row_num = asset['creative_id'], asset['row_num']
    print(f"ERROR - {row_num} ID: {creative_id}: {e}, GCS_Uploader")
    gcs_obj = {'row_num': row_num,
               'creative_id': creative_id,
               'error': str(e),
               'stage': 'GCS Uploader'}
    return gcs_obj

Overwriting gcs_helper.py


In [ ]:
%%writefile image_helper.py

from utils import vision_client
from google.cloud import vision
from google.protobuf.json_format import MessageToDict
import re
import time

def get_annotation_features():
  features = [{"type_": vision.Feature.Type.TEXT_DETECTION},
              {"type_": vision.Feature.Type.IMAGE_PROPERTIES},
              {"type_": vision.Feature.Type.SAFE_SEARCH_DETECTION},
              {"type_": vision.Feature.Type.LABEL_DETECTION},
              {"type_": vision.Feature.Type.LOGO_DETECTION},
              {"type_": vision.Feature.Type.FACE_DETECTION},
              {"type_": vision.Feature.Type.OBJECT_LOCALIZATION}]
  return features


def extract_image_metadata(gcs_obj):
  try:
    global vision_client
    time.sleep(10)
    frame_id, frame_url = gcs_obj['frame_id'], gcs_obj['frame_url']
    gcs_prefix = "https://storage.cloud.google.com"
    image = {"source": {"image_uri": frame_url}}
    features = get_annotation_features()
    response = vision_client.annotate_image({"image": image, "features": features})
    message = MessageToDict(response._pb)
    endpoint_resp = lambda k: message[k] if k in message else None
    raw_text = endpoint_resp("textAnnotations")
    face_annotations = endpoint_resp("faceAnnotations")
    if raw_text:
      if "locale" in raw_text[0]:
        raw_text = raw_text[1:]
      else:
        for i, text in raw_text.items():
          if "locale" in text:
            del raw_text[i]["locale"]

    if face_annotations:
      for idx, item in enumerate(face_annotations):
        for i, value in face_annotations[idx].items():
          if i == "landmarks":
            for k, j in enumerate(value):
              face_annotations[idx][i][k]["type"] = str(j["type"])
              
    metadata = {
      "frame_id": frame_id,
      "frame_url": gcs_prefix + frame_url[4:],
      "text_annotations": raw_text,
      "image_properties_annotation": endpoint_resp("imagePropertiesAnnotation"),
      "safe_search_annotation": endpoint_resp("safeSearchAnnotation"),
      "label_annotations": endpoint_resp("labelAnnotations"),
      "logo_annotations": endpoint_resp("logoAnnotations"),
      "face_annotations": face_annotations,
      "localized_object_annotations": endpoint_resp("localizedObjectAnnotations")
    }
    #print("Image Metadata: ", frame_id)
    return metadata
  except Exception as e:
    frame_id = gcs_obj['frame_id']
    row_num = re.findall(r".+_",str(frame_id))[0][:-1]
    print(f"ERROR - {row_num} ID: {frame_id}: {e}, Image Annotation")
    metadata = {'row_num': row_num,
                'frame_id': frame_id,
                'error': str(e),
                'stage': 'Image Metadata Extraction'}
    return metadata

Overwriting image_helper.py


In [ ]:
%%writefile video_helper.py

from google.cloud import videointelligence
from google.protobuf.json_format import MessageToDict
from utils import params, credentials
from utils import vid_client, vid_config, video_context, vid_features
import time



def process_text(text_annotations):
  filter_seg = lambda x: {"confidence": x["confidence"], "segment": x["segment"],
                          "frames": x["frames"]} 
  filter_text = lambda x: {"text": x["text"],
                           "segments": [filter_seg(s) for s in x["segments"]]}
  processed = [filter_text(text) for text in text_annotations]
  return processed

def process_labels(label_annotations):
  filter_label = lambda x: {'entity':x['entity'],'segments':x['segments']}
  processed = [filter_label(label) for label in label_annotations]
  return processed

def process_speech(speech):
  filter_speech = lambda x: {"alternatives": x["alternatives"]}
  processed = [filter_speech(x) for x in speech if x["alternatives"] != [{}] ]
  return processed

def split_metadata(all_metadata):
  annotations, tran = all_metadata # tran for transcriptions
  get_one_metadata = lambda x: annotations[x] if x in annotations else []
  filtering_endpoints = ["explicitAnnotation", "objectAnnotations",
                         "textAnnotations", "segmentLabelAnnotations",
                         "shotLabelAnnotations", "shotAnnotations"] 
  values = [get_one_metadata(e) for e in filtering_endpoints]
  speech = tran['speechTranscriptions'] if 'speechTranscriptions' in tran else []
  values.append(speech)
  return values

def extract_video_metadata(gcs_obj):
  try:
    time.sleep(10)
    gcs_prefix = "https://storage.cloud.google.com"
    creative_id, creative_url = gcs_obj['creative_id'], gcs_obj['gcs_url']
    row_num = gcs_obj['row_num']
    global vid_client, vid_config, video_context, vid_features
    job = vid_client.annotate_video(input_uri=creative_url,
                                    features=vid_features,
                                    video_context=video_context)
    result = job.result()
    message = MessageToDict(result)
    all_metadata = message['annotationResults']
    if len(all_metadata) != 2:
      print("ERROR: ", row_num, "Invalid format.", "Video Analysis")
      metadata = {'row_num': row_num,
                  'creative_id': creative_id,
                  'error': "Invalid format.",
                  'stage': 'Video Metadata Extraction'}
      return metadata
    meta_values = split_metadata(all_metadata)
    explicit, objects, text, segment_labels = meta_values[:4]
    shot_labels, shots, speech = meta_values[4:]
    row = {'row_num': row_num,
           'creative_id': creative_id,
           'creative_url': gcs_prefix + creative_url[4:],
           'text_annotations': process_text(text),
           'segment_label_annotations': process_labels(segment_labels),
           'shot_label_annotations': process_labels(shot_labels),
           'speech_transcriptions':process_speech(speech),
           'explicit_annotation': explicit,
           'object_annotations': objects,
           'shot_annotations': shots}
    
    print("Finished Video Analysis: ", row_num)
    return row
  except Exception as e:
    row_num, creative_id = gcs_obj['row_num'], gcs_obj['creative_id']
    print("ERROR: ", row_num, e, "Video Analysis")
    metadata = {'row_num': row_num,
                'creative_id': creative_id, 
                'error': str(e),
                'stage': 'Video Metadata Extraction'}
    return metadata

Overwriting video_helper.py


In [ ]:
%%writefile main.py
import requests
import warnings
import time
import os
import ndjson
from retrying import retry
from concurrent.futures import ThreadPoolExecutor
from google.cloud import storage
from google.cloud import vision
from google.cloud import bigquery
from google.oauth2 import service_account
from google.protobuf.json_format import MessageToDict
from utils import params
import bq_helper
import gcs_helper
import image_helper
import video_helper
import re


warnings.simplefilter(action="ignore")

def has_noretry_err(err_message):
  error_keyphrases = ['404 ', 'Invalid format']
  for e in error_keyphrases:
    if re.search(e, err_message):
      return True
  return False


def upload_and_annotate(asset):
  global params
  job_type = params['job_type']
  gcs_obj = gcs_helper.write_to_gcs(asset)
  if 'error' in gcs_obj:
    return gcs_obj
  if job_type == "image":
    frames = gcs_obj["frames"]
    frames_meta = [image_helper.extract_image_metadata(f) for f in frames]
    for f in frames_meta:
      if 'error' in f:
        error = {'row_num': gcs_obj["row_num"],
                'creative_id': gcs_obj["creative_id"],
                'error': f['error'],
                'stage': 'Image Metadata'}
        return error
    metadata = {'row_num': gcs_obj["row_num"],
                'creative_id': gcs_obj["creative_id"],
                'gcs_url': gcs_obj["gcs_url"],
                'frames': frames_meta}
  else:
    metadata = video_helper.extract_video_metadata(gcs_obj)
  return metadata

def get_read_query():
  project = params['gcp_project']
  input_dataset = params['bq_input_dataset'] 
  input_table = params['bq_input_tablename']
  sql = f"SELECT * FROM `{project}.{input_dataset}.{input_table}`"
  return sql

def print_summary_stats(i, n_valid, n_errors, n_retry):
  print(f"After try {i},")
  print(f"Valid results: {n_valid}")
  print(f"No. of all errors: {n_errors}")
  print(f"No. of retriable errors: {n_retry}")

def main():
  global params
  job_type, asset_source = params['job_type'], params['asset_source']
  if asset_source == "gcs": 
    input_data = gcs_helper.read_from_gcs()
    input_dataset = params['bq_input_dataset'] 
    input_table = params['bq_input_tablename']
    bq_helper.write_to_bq(input_dataset, input_table, input_data)
  base_sql = get_read_query()
  sql = base_sql + f" LIMIT {params['limit']}" if 'limit' in params else base_sql
  max_workers = 300 if job_type == "image" else 15
  NUM_RETRIES = 3 if job_type == "image" else 10
  executor =  ThreadPoolExecutor(max_workers=max_workers)
  valid_res, noretry_error_ids = [], []
  for i in range(1, NUM_RETRIES+1):
    asset_urls = bq_helper.read_from_bq(sql)
    results = list(executor.map(upload_and_annotate, asset_urls))
    #results = list(map(upload_and_annotate, asset_urls))
    valid_res = valid_res + [result for result in results if 'error' not in result]
    err_rows = [result for result in results if 'error' in result]
    all_nums = [r['row_num'] for r in err_rows]
    noretry = [r['row_num'] for r in err_rows if has_noretry_err(r['error'])]
    noretry_error_ids = noretry_error_ids + noretry
    retry_nums = list(set(all_nums) - set(noretry))
    n_valid, n_errors, n_retry = len(valid_res), len(err_rows), len(retry_nums)
    print_summary_stats(i, n_valid, n_errors, n_retry)
    if not retry_nums:
      break
    sql = base_sql+" WHERE row_num IN "+str(tuple(retry_nums))
    sql = sql[:-2]+")" if sql[-2:] == ",)" else sql
    time.sleep(60)
  print(f"Total no. of non-retriable errors: {len(noretry_error_ids)}")
  endpoints = bq_helper.get_endpoints(job_type)
  output_dataset = params['bq_output_dataset'] 
  for endpoint in endpoints:
    filtered = [bq_helper.filter_output(v, endpoint) for v in valid_res]
    executor.submit(bq_helper.write_to_bq, output_dataset, endpoint, filtered)
  executor.shutdown()


#time.sleep(60)
start = time.time()
main()
end = time.time()
print(f"The pipeline took {end-start} seconds.")

Overwriting main.py


### Run below cell to start the job and get creative data

In [ ]:
!python3 main.py

After try 1,
Valid results: 52
No. of all errors: 0
No. of retriable errors: 0
Total no. of non-retriable errors: 0
Loaded localized_object_annotations table
Loaded face_annotations table
Loaded label_annotations table
Loaded safe_search_annotation table
Loaded image_properties_annotation table
Loaded text_annotations table
Loaded logo_annotations table
The pipeline took 1380.2823300361633 seconds.
